## LGB Submission and metric breakdown

MARKET DATA ONLY - This is only for submission


__Notes:__ However, you will not get a score back unless you commit the kernel and submit to Kaggle. Still need to do this

In [ ]:
from shutil import copyfile


# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/supporting-files-sigma/sigma_libs.py", dst = "../working/sigma_libs.py")
copyfile(src = "../input/supporting-files-sigma/sigma_1.py", dst = "../working/sigma_1.py")
copyfile(src = "../input/supporting-files-sigma/preproc2.py", dst = "../working/preproc2.py")
# import all our functions
from sigma_libs import *
from sigma_1 import *
from preproc2 import *
tqdm.pandas() 

In [ ]:
df_market, df_news = env.get_training_data()
# MARKET DATA ONLY

In [ ]:
# MARKET ONLY
del df_news
gc.collect()

In [ ]:
# SOME BASIC ENGINEERING

def train_feats(dframe):
    # could do list comp
    train_feat = []
    for col in dframe.columns:
        if col not in ['time', 'assetCode', 'universe', 'returnsOpenNextMktres10']:
            train_feat.append(col)
    return train_feat

def preproc(dframe):
    
   
    dframe['avg'] = (dframe.close + dframe.open)/2
    dframe['pricevolume'] = dframe.volume * df_market.close
    dframe.drop(['assetName'], axis=1, inplace=True)
    
    # REDUCED - arbitrary    
    start = datetime(2012, 1, 1, 0, 0, 0).date()
    df_= dframe.loc[dframe['time'].dt.date >= start].reset_index(drop=True)
    # This form of date is better for setting the ranges later
    df_['time'] = df_.time.dt.strftime("%Y%m%d").astype(int)

    return df_


In [ ]:
# MINI PIPELINE
df_m = df_market.copy() 
df = preproc(df_m)
train_feat = train_feats(df)

In [ ]:
# SET RANGES
# There should be a better way to do this. But this is quick and based off
# percentiles. I would initially go be dates to set the ranges. Doesn't need 
# to be an int. Could include this in a Time_ class.
dates = df['time'].unique()

train_range = range(len(dates))[:int(0.85*len(dates))]
val_range = range(len(dates))[int(0.85*len(dates)):]

# Convert targets to binary
target = ['returnsOpenNextMktres10']
df[target[0]] = (df[target[0]] > 0).astype(int)

__So lightgbm is awesome, fast, and efficient. Time to drop xgb and make this your go-to model__


Preload the data into the model as numpy arrays. It has a zillon parameters and other options. The validation set is embedded into the model. Allows the model to train against the validation set and then select the best iteration. I'd have to research this more. Typically, validations are only used like this for NN. It's not using CV, just the validation set.

In [2]:
import lightgbm as lgb
# THIS INSTATIATES IT 

In [ ]:
# KINDA LOOKS CRAPPY. THIS JUST SETS THE TRAINING/VAL SETS WITH PREDICTORS
# train data - numpys
X_train = df[train_feat].fillna(0).loc[df['time'].isin(dates[train_range])].values
Y_train = df[target].fillna(0).loc[df['time'].isin(dates[train_range])].values

# GETS INTERESTING -> INPUTS DATA INTO INSTANCE
lgb_train = lgb.Dataset(X_train, Y_train[:,0])
print(X_train.shape, Y_train.shape)

In [ ]:
# validation data - numpys
X_v = df[train_feat].fillna(0).loc[df['time'].isin(dates[val_range])].values
Y_v = df[target].fillna(0).loc[df['time'].isin(dates[val_range])].values

lgb_val = lgb.Dataset(X_v, Y_v[:,0]),
print(X_v.shape, Y_v.shape)

In [ ]:

param = {"objective" : "binary",
          "metric" : "binary_logloss",
          "verbosity" : -1 }

# TONS OF HYPERPARAMENTERS. THIS FIT FUNC USES THE VAL SET TO TRAIN AND 
# SELECT THE BEST ITERATION

model = lgb.train(param, lgb_train, valid_sets=lgb_val, 
                  early_stopping_rounds=10) 
                  

## Daily test data

Need to explore this more. But I think we get the picture. Initially, the set that I looked at was around 2k entries. This is somwhat consistent with the number of assets for a given day. My question: how many days are we talking about here? We know the training sets cut out at the start of 2018 (confirm). I know this is two phased. We need a better understanding of the testing processes. This is synthetic data, so phase 2 uses real data that is current?  

market _train_ data $\approx 40.7$ million (feb2007 to dec2016)  

market _test_ data $\approx 1800$ (jan2017). __I believe this is per day. so in actuality, if the test set covers 2017, we're looking at 657K. Double that if we include 2018__  

In [ ]:
# SO RUN THIS FIRST, IT WILL RETURN THE 3 OBJECTS (DFRAMES) FOR EACH DAY
days = env.get_prediction_days()
# THE AUTHOR USED THIS TO GRAB ONE DAY FOR ANALYSIS
# (market_obs_df, news_obs_df, predictions_template_df) = next(days)

In [47]:
# NEED TO LOOP THROUGH EACH DAY AND RENDER PREDICTIONS

for market, news, pred_df in days:
    # preprocess
    X = preproc(market)
    # predictors
    X_test = X[train_feat].fillna(0).values
    # pred vec -> *2 -1 (need to check this math)
    preds = model.predict(X_test, num_iteration=model.best_iteration) * 2 - 1
    # create dframe for my preds
    df_pred = pd.DataFrame({'ast':X['assetCode'],'conf':preds})
    # if my predicted asset is in the provided kaggle dframe, then
    # set the kaggle feat [con val] == to my prediction
    pred_df['confidenceValue'][pred_df['assetCode'].isin(df_pred.ast)] = df_pred['conf'].values
    # upload predictions 
    env.predict(pred_df)

env.write_submission_file()

Your submission file has been saved. Once you `Commit` your Kernel and it finishes running, you can submit the file to the competition from the Kernel Viewer `Output` tab.


Not sure about how many days. But if you like, you can break the _daily_ dframes down. Nothing special, but need to inspect further. 

In [ ]:
X = preproc(market_obs_df)

X_test = X[train_feat].fillna(0).values

preds = model.predict(X_test, num_iteration=model.best_iteration) * 2 - 1

preds_toDF = pd.DataFrame({'ast':X['assetCode'],'conf':preds})

preds_toDF.shape

predictions_template_df['confidenceValue'][predictions_template_df['assetCode'].isin(preds_toDF.ast)] = preds_toDF['conf'].values

predictions_template_df.shape

env.predict(predictions_template_df)

env.write_submission_file()



In [ ]:
(market_obs_df, news_obs_df, predictions_template_df) = next(days)